In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.textract import parse_textract_datetime
from lib.explore import print_df
from lib.match import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)
from clean.baton_rouge_pd_cprr import clean_18
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19

In [2]:
df17 = clean_csd_17()
df17

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../lib/clean.py:156: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\d\.]", "").astype("float64")


,year,employee_id,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,...,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,dept_year,rank_year,perhist_uid
5659,2008,456780,tate,joshua,j,5050001,police department-operation serv bureau,5005,police officer,5,...,21,,,,Baton Rouge CSD,0004aa9e5a7fe44d2e8746aeff1d2e97,NaN,NaN,NaN,751acb62e78bb7425c21905be58ba5cb
3587,2008,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,NaN,NaN,NaN,60d5c258e8c9f87b0dc2772f64be1a94
3588,2009,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,2009.0,NaN,NaN,9c3c66f7579e5d5e7aac83b75d5d2917
3589,2010,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,2010.0,NaN,NaN,42577dca91d1d267ea8e2931165d45b0
3590,2011,387088,bailey,jana,m,5010001,police department-administration,110435,senior legal specialist,0,...,2,,,,Baton Rouge CSD,000d35f47a90a410d73ec0ddf6dd943d,2011.0,NaN,NaN,aae94c8b8dfbbbfecc642d4f4ca873c1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,2013,383910,royal,althea,p,5050001,police department-operation serv bureau,110090,clerical specialist,0,...,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2013.0,NaN,NaN,4d1fc729629d4313dccddc2fa310e4f6
3468,2014,383910,royal,althea,p,5050001,police department-operation serv bureau,110090,clerical specialist,0,...,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2014.0,NaN,NaN,574eedcf575f347a9957fa343297b1da
7553,2015,383910,royal,althea,p,5050001,police department-operation serv bureau,110132,senior clerical specialist,0,...,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2015.0,NaN,2015.0,ea060d245168a155fc2f6f31d7556ff7
8876,2016,383910,royal,althea,p,5050001,police department-operation serv bureau,110132,senior clerical specialist,0,...,7,,,,Baton Rouge CSD,ffc4c92c0507e19003f3ffbc8a347aba,2016.0,NaN,NaN,fc857dd518dd692ee0845cf084ef1f22


In [3]:
df19 = clean_csd_19()
df19

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../lib/clean.py:156: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\d\.]", "").astype("float64")
../lib/clean.py:150: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.strip().str.lower()\


,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,annual_salary,...,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year,perhist_uid
404,2018,sawyer,laterial,d,5020,police-uniform patrol bureau,5005,police officer,active,43884.51,...,18,,,,Baton Rouge CSD,00470dc28b790079ea4003c6ea9d9a6d,NaN,NaN,NaN,f7500576ef9590a2c8417c889653da3b
1332,2019,sawyer,laterial,d,5020,police-uniform patrol bureau,5005,police officer,active,46039.68,...,18,,,,Baton Rouge CSD,00470dc28b790079ea4003c6ea9d9a6d,2019.0,NaN,NaN,0d34f9617d960b75841b84268915272d
684,2018,blake,brian,r,5020,police-uniform patrol bureau,5010,police sergeant,active,72848.05,...,9,,,,Baton Rouge CSD,007f09c47926f096be2ef8013c798c19,NaN,NaN,NaN,3c1ec2d03bf399a63b20deb203e35a1a
1591,2019,blake,brian,r,5020,police-uniform patrol bureau,5010,police sergeant,active,76664.95,...,9,,,,Baton Rouge CSD,007f09c47926f096be2ef8013c798c19,2019.0,NaN,NaN,6b1a194f0c1d21178ab54c97214f19a7
678,2018,ned,larry,m,5020,police-uniform patrol bureau,5010,police sergeant,active,56126.86,...,4,,,,Baton Rouge CSD,00e895256b8e81fdcd52c67969411cae,NaN,NaN,NaN,a9e7a9ba123a0c90731b6600797158f7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,2019,brooks,charlene,j,5040,police-operational serv bureau,5235,police fingerprint tech i,inactive,7345.06,...,21,2018,11,30,Baton Rouge CSD,fe48657a1e3f89951ce2dbf0112cd8e3,2019.0,NaN,NaN,2184248d6d16599ae475fe856444928d
75,2018,pointer,sheldon,d,5020,police-uniform patrol bureau,5005,police officer,inactive,63382.81,...,27,2018,11,1,Baton Rouge CSD,ff3e06c89dee3ec36c1f6da1624a0b4b,NaN,NaN,NaN,58951a673d42945028946bd6e9cdfe85
199,2018,williams,liza,s,5020,police-uniform patrol bureau,5005,police officer,active,52178.13,...,29,,,,Baton Rouge CSD,ff858ea814c23620dcc3cfef85dded8f,NaN,NaN,NaN,440b07e326af1198d456eba0bc447675
1147,2019,williams,liza,s,5020,police-uniform patrol bureau,5005,police officer,active,52728.62,...,29,,,,Baton Rouge CSD,ff858ea814c23620dcc3cfef85dded8f,2019.0,NaN,NaN,99ed3dd06b1aabcfa593ceddcfd33e88


In [6]:
df19[df19.perhist_uid.duplicated(keep=False)]

,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,annual_salary,...,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year,perhist_uid
